Old integrations (random and naireen) use an old REBOUND commit, but celmech needs later REBOUND versions that accept jacobi_masses to sim.calculate_orbits(). Pulling the code into training_functions.py and checking we get same thing as with later versions of REBOUND

In [1]:
import rebound

In [2]:
sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-3, P=1., e=0.2)
sim.add(m=1.e-3, P=1.5, e=0.1)
sim.add(m=1.e-3, P=4., e=0.2)
sim.move_to_com()
ps = sim.particles

In [3]:
o = sim.calculate_orbits(jacobi_masses=True)
for orbit in o:
    print(orbit)

<rebound.Orbit instance, a=0.2937817275050218 e=0.20000000000000023 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
<rebound.Orbit instance, a=0.38556236008350964 e=0.10110000000000031 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
<rebound.Orbit instance, a=0.7430052823165784 e=0.20240000000000005 inc=0.0 Omega=0.0 omega=0.0 f=0.0>


In [4]:
print(ps[3].calculate_orbit())

<rebound.Orbit instance, a=0.740776266469629 e=0.20000000000000043 inc=0.0 Omega=0.0 omega=0.0 f=0.0>


In [5]:
from celmech.transformations import masses_to_jacobi
mjac, Mjac = masses_to_jacobi([p.m for p in ps])

In [6]:
primary = sim.calculate_com(last=3)
print(primary)

<rebound.Particle object, m=1.0019999999999998 x=-0.0005908484677723864 y=0.0 z=0.0 vx=0.0 vy=-0.0014208611287366355 vz=0.0>


In [7]:
primary.m = Mjac[3]-ps[3].m

In [8]:
print(ps[3].calculate_orbit(primary=primary))

<rebound.Orbit instance, a=0.7430052823165784 e=0.20240000000000005 inc=0.0 Omega=0.0 omega=0.0 f=0.0>


In [9]:
import numpy as np
from celmech import Poincare

mjac, Mjac = masses_to_jacobi([p.m for p in ps])
pvars = Poincare(sim.G)
for i in range(1, sim.N-sim.N_var):
    M = Mjac[i]
    m = mjac[i]
    primary = sim.calculate_com(last=i)
    primary.m = Mjac[i]-ps[i].m
    o = ps[i].calculate_orbit(primary=primary)
    print(o)
    sLambda = np.sqrt(sim.G*M*o.a)
    sGamma = sLambda*(1.-np.sqrt(1.-o.e**2))
    pvars.add(m=m, sLambda=sLambda, l=o.l, sGamma=sGamma, gamma=-o.pomega, M=M)
pvars.average_synodic_terms()
for p in pvars.particles:
    print(p.sLambda, p.sGamma, p.l, p.gamma, p.M, p.m)

<rebound.Orbit instance, a=0.2937817275050218 e=0.20000000000000023 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
<rebound.Orbit instance, a=0.38556236008350964 e=0.10110000000000031 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
<rebound.Orbit instance, a=0.7430052823165784 e=0.20240000000000005 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
nan nan nan nan nan nan
0.5395034928159745 0.010956428282117242 3.1539813412005785e-08 -0.0 1.001 0.0009990009990009992
0.6237226153039576 0.0031831014781202654 1.3394653797149654e-08 -0.0 1.0009990009990009 0.0009990019960079842
0.8627155060650645 0.017849304725275024 2.9112592620311125e-08 -0.0 1.0009980039920159 0.0009990029910269195


In [10]:
o = sim.calculate_orbits(jacobi_masses=True)
for orbit in o:
    print(orbit)

<rebound.Orbit instance, a=0.2937817275050218 e=0.20000000000000023 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
<rebound.Orbit instance, a=0.38556236008350964 e=0.10110000000000031 inc=0.0 Omega=0.0 omega=0.0 f=0.0>
<rebound.Orbit instance, a=0.7430052823165784 e=0.20240000000000005 inc=0.0 Omega=0.0 omega=0.0 f=0.0>


In [11]:
pvars = Poincare.from_Simulation(sim)#, average=False)
#pvars.average_synodic_terms()
for p in pvars.particles:
    print(p.sLambda, p.sGamma, p.l, p.gamma, p.M, p.m)

nan nan nan nan nan nan
0.5395034928159745 0.010956428282117242 3.1539813412005785e-08 -0.0 1.001 0.0009990009990009992
0.6237226153039576 0.0031831014781202654 1.3394653797149654e-08 -0.0 1.0009990009990009 0.0009990019960079842
0.8627155060650645 0.017849304725275024 2.9112592620311125e-08 -0.0 1.0009980039920159 0.0009990029910269195


In [12]:
from celmech import Andoyer

avars = Andoyer.from_Simulation(sim, j=3, k=1, a10=ps[1].a, i1=1, i2=2)
print(avars.Z, avars.phi)

0.04853498504436155 3.141592630694128


In [13]:
from celmech import Andoyer
from celmech.transformations import masses_to_jacobi
import numpy as np
from celmech import Poincare

mjac, Mjac = masses_to_jacobi([p.m for p in ps])
pvars = Poincare(sim.G)
for i in range(1, sim.N-sim.N_var):
    M = Mjac[i]
    m = mjac[i]
    primary = sim.calculate_com(last=i)
    primary.m = Mjac[i]-ps[i].m
    o = ps[i].calculate_orbit(primary=primary)
    sLambda = np.sqrt(sim.G*M*o.a)
    sGamma = sLambda*(1.-np.sqrt(1.-o.e**2))
    pvars.add(m=m, sLambda=sLambda, l=o.l, sGamma=sGamma, gamma=-o.pomega, M=M)
pvars.average_synodic_terms()
avars = Andoyer.from_Poincare(pvars, j=3, k=1, a10=ps[1].a, i1=1, i2=2)
print(avars.Z, avars.phi)

0.04853498504436155 3.141592630694128
